In [1]:
import os
from typing import Tuple
from tensorflow.python.keras.utils.generic_utils import Progbar
from tf_agents.environments import suite_gym, parallel_py_environment
from tf_agents.environments import tf_py_environment
from tf_agents.metrics import tf_metrics
from tf_agents.replay_buffers import tf_uniform_replay_buffer, episodic_replay_buffer
from tf_agents.trajectories.trajectory import Trajectory
from tf_agents.networks import actor_distribution_network
from tf_agents.drivers import dynamic_episode_driver, dynamic_step_driver
from tf_agents.policies import random_tf_policy
import tensorflow as tf
import numpy as np
from tf_agents.utils import common
from reinforcement_learning.sac_training import NumberOfSafetyViolations
from tf_agents.trajectories import time_step as ts, policy_step, trajectory
from reinforcement_learning import sac_training
from reinforcement_learning import labeling_functions
labeling_function = labeling_functions['BipedalWalker-v2']
tf.config.set_visible_devices([], 'GPU')  #  allows testing during training

In [2]:
py_env = suite_gym.load('BipedalWalker-v2')
py_env.render(mode='human')
py_env.reset()
tf_env = tf_py_environment.TFPyEnvironment(py_env)
tf_env.time_step_spec()


TimeStep(step_type=TensorSpec(shape=(), dtype=tf.int32, name='step_type'), reward=TensorSpec(shape=(), dtype=tf.float32, name='reward'), discount=BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)), observation=BoundedTensorSpec(shape=(24,), dtype=tf.float32, name='observation', minimum=array(-3.4028235e+38, dtype=float32), maximum=array(3.4028235e+38, dtype=float32)))

In [3]:
import variational_action_discretizer

vae_mdp = variational_action_discretizer.load(
    #'../../saves/BipedalWalker-v2/models/vae_LS14_MC1_ER10.0-decay=7e-05-min=0_KLA0.0-growth=5e-05_TD0.66-0.50_2e-06-1e-06_seed=70421/policy/action_discretizer/LA5_MC1_ER10.0-decay=7e-05-min=0_KLA0.0-growth=5e-05_TD0.25-0.17_2e-06-1e-06_params=full_vae_optimization-relaxed_state_encoding/base',
    '../../saves/BipedalWalker-v2/models/vae_LS14_MC1_ER10.0-decay=1e-05-min=0_KLA0.0-growth=5e-05_TD0.67-0.50_1e-06-2e-06_seed=70421/policy/action_discretizer/LA5_MC1_ER10.0-decay=1e-05-min=0_KLA0.0-growth=5e-05_TD0.25-0.17_1e-06-2e-06_params=full_vae_optimization-relaxed_state_encoding/base',
    step=930000
)
print("VAE MDP loaded")

vae_mdp.eval_policy(eval_env=py_env,
                    labeling_function=labeling_function,
                    num_eval_episodes=20,
                    render=True)

VAE MDP loaded


KeyboardInterrupt: 